In [ ]:
# set module
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')
#경로 설정
import os
new_dir = "/content/drive/MyDrive/기상청/데이터/지면온도"
os.chdir(new_dir)  # 현재 작업 디렉토리 변경
current_dir = os.getcwd()
print("현재 작업 디렉토리:", current_dir)

In [ ]:
# 데이터 읽어오기
data_train = pd.read_csv("./surface_tp_train.csv")
data_test= pd.read_csv("./surface_tp_test.csv")

In [ ]:
## column 명 추출
data_train.drop('Unnamed: 0', axis = 1, inplace = True)
data_train.drop('...1', axis = 1, inplace = True)
data_test.drop('Unnamed: 0', axis = 1, inplace = True)

## 시각화 진행

In [ ]:
## 대푯값 확인 -> train
data_train.describe()

In [ ]:
data_train.info()

In [ ]:
## 제거 : -99, -99.9, -999

import pandas as pd
import numpy as np

class data_preprocessing():
  def __init__(self, data, test):
    self.data = data
    self.na_counts = None
    self.list_na = None
    self.ratio_result = None
    self.list_columns = None
    self.test = test

  ## na 개수 발견
  def count_na_per_column(self):
    special_values = [-99, -99.9, -999]
    list_counts = []
    for col in self.data.columns:
        counts = self.data[col].isin(special_values).sum()
        list_counts.append(counts)
    self.list_na = list_counts
    return self.list_na

  def divide_list_elements(self, lst, divisor):
      divided_lst = [x / divisor for x in lst]
      return divided_lst

  ## na 개수 전체 데이터 중 비율 세기
  def ratio(self):
    self.ratio_result = self.divide_list_elements(self.list_na, len(self.data))
    return self.ratio_result

  def remove_column(self):
    self.list_columns = []
    # 역순으로 반복문 실행하여 열을 삭제
    for i in range(len(self.ratio_result)-1, -1, -1):
        if self.ratio_result[i] >= 0.4:
          self.list_columns.append(self.data.columns[i])
            # print(self.ratio_result[i])
            # print(i)
          self.data = self.data.drop(self.data.columns[i], axis=1)
    return self.data

  def remove_columns_test(self):
    data = self.data
    for i in self.list_columns:
      ## . 기준으로 마지막 부분 없애고,
      new_col = i.replace("train", "test")
      # print(new_col)
      try: ## train만 있는 열이 있을 경우에는 pass 하자
        data = data.drop(new_col, axis=1)
      except:
        pass
    return data


  def result(self, train_or_test):
    if train_or_test == "train":
      # self.replace_99()
      self.count_na_per_column()
      self.ratio()
      return self.remove_column()
    else:
      self.data = self.test
      # self.replace_99() ## null 로 변경
      return self.remove_columns_test() ## train에서 제거되는 행으로 제거

In [ ]:
dp= data_preprocessing(data_train, data_test)
result_train = dp.result('train')
result_train
result_test = dp.result("test")
result_test

In [ ]:
result_train.dropna(inplace = True)
result_train.info()

In [ ]:
# result_train['hh'] = result_train['surface_tp_train.mmddhh'].astype(int).astype(str).str[-2:]
# result_train['dd'] = result_train['surface_tp_train.mmddhh'].astype(int).astype(str).str[-4:-2]
# result_train['mm'] = result_train['surface_tp_train.mmddhh'].astype(int).astype(str).str[:-4]
# year_mapping = {'A': 2001, 'B': 2002, 'C': 2003, 'D': 2004, 'E': 2005, 'F': 2006}
# result_train['new_year'] = result_train['surface_tp_train.year'].map(year_mapping)
# result_train['mm'] = result_train['mm'].apply(lambda x: x.zfill(2))
# result_train['date'] = pd.to_datetime(result_train['new_year'].astype(str) + result_train['mm'] + result_train['dd'] + result_train['hh'], format='%Y%m%d%H')

def adding_date(data_original, season): # season: all, spring, summer, fall, winter 계절별로 넣기 가능
  data = data_original
  data['hh'] = data['surface_tp_train.mmddhh'].astype(int).astype(str).str[-2:]
  data['dd'] = data['surface_tp_train.mmddhh'].astype(int).astype(str).str[-4:-2]
  data['mm'] = data['surface_tp_train.mmddhh'].astype(int).astype(str).str[:-4]
  year_mapping = {'A': 2001, 'B': 2002, 'C': 2003, 'D': 2004, 'E': 2005, 'F': 2006}
  data['new_year'] = data['surface_tp_train.year'].map(year_mapping)
  data['mm'] = data['mm'].apply(lambda x: x.zfill(2))
  data['date'] = pd.to_datetime(data['new_year'].astype(str) + data['mm'] + data['dd'] + data['hh'], format='%Y%m%d%H')

  spring_month_list = ['02', '03', '04']
  summer_month_list = ['05', '06', '07']
  fall_month_list = ['08', '09', '10']
  winter_month_list = ['11', '12', '01']

  if season == "all":
    pass
  elif season == "spring":
    filtered_data = data[data['mm'].isin(spring_month_list)]
  elif season == "summer":
    filtered_data = data[data['mm'].isin(summer_month_list)]
  elif season == "fall":
    filtered_data = data[data['mm'].isin(fall_month_list)]
  elif season == "winter":
    filtered_data = data[data['mm'].isin(winter_month_list)]
  else:
    raise ValueError("not season value")

  # data = filtered_data.drop(["surface_tp_train.year", 'surface_tp_train.mmddhh', 'hh', 'dd', 'mm', 'new_year'], axis = 1)
  data = filtered_data.drop(["surface_tp_train.year", 'surface_tp_train.mmddhh'], axis = 1)
  # data["datetime"] = pd.to_datetime(data["date"])  # 문자열을 날짜/시간 유형으로 변환
  ## datetime 순으로 index 생성
  data.reset_index(inplace=True)
  data.index = data.index.astype(int)
  return data

### 지역 별로 데이터를 쪼개주는 함수

def region_category(data, region_number): # region_number는 자기가 알아서 넣기로

  # 주어진 열을 기준으로 데이터 프레임을 필터링하고 값 종류별로 데이터를 쪼갬
  unique_values = data['surface_tp_train.stn'].unique()  # 주어진 열의 고유한 값들을 추출

  if region_number in unique_values:
    result = data[data['surface_tp_train.stn'] == region_number]  # 주어진 값에 해당하는 데이터만 필터링하여 저장
  else:
    raise ValueError("not in region number")

  return result


def change_columns(data):
  data.columns = data.columns.str.replace(".", "_")


def change_to_category(data, columns_name):
  data[columns_name] = data[columns_name].astype(str).astype('category')
  return data


In [ ]:
result_train_spring = adding_date(result_train, "summer")
result_train_spring.columns

## region 별로 나누기
train_region_1_spring = region_category(result_train_spring, 1)
train_region_2_spring = region_category(result_train_spring, 2)
train_region_3_spring = region_category(result_train_spring, 3)
train_region_4_spring = region_category(result_train_spring, 4)
train_region_5_spring = region_category(result_train_spring, 5)
train_region_6_spring = region_category(result_train_spring, 6)
train_region_7_spring = region_category(result_train_spring, 7)
train_region_8_spring = region_category(result_train_spring, 8)
train_region_9_spring = region_category(result_train_spring, 8)
train_region_10_spring = region_category(result_train_spring, 10)

In [ ]:
## category 명 변경;
def category_changing(data):
  list_columns = ["surface_tp_train_stn", "new_year", "mm", "dd", "hh", "surface_tp_train_ww"]
  for column in list_columns:
      data = change_to_category(data, column)

## time index  제작:
def indexing(data):
  ## column 이름에 . 제거
  change_columns(data)
  data.drop("index", axis = 1, inplace=True)
  ## sort 하고 타임 index 생성
  data.sort_values("date", ascending = True, inplace = True)
  data['time_idx'] = range(len(data))
  return data

## date drop:
def drop_date(data):
  data.drop("date", axis = 1, inplace = True)

## main 실행
list_data = [train_region_1_spring, train_region_2_spring, train_region_3_spring, train_region_4_spring, train_region_5_spring
             ,train_region_6_spring,train_region_7_spring,train_region_8_spring,train_region_9_spring, train_region_10_spring]

for data in list_data:
  change_columns(data)
  category_changing(data)
  data = indexing(data)
  drop_date(data)

In [ ]:
new_directory = "/content/drive/MyDrive/기상청 (1)"
os.chdir(new_directory)
root_directory = os.getcwd()
root_directory

In [ ]:
train_region_1_spring

In [ ]:

!nvidia-smi

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

## LSTM 코드 구현

### 변수
<div>
  <table>
    <tr>
      <th>Column</th>
      <th>Description</th>
      <th>Data Type</th>
    </tr>
    <tr>
      <td>surface_tp_train.stn</td>
      <td>지역번호</td>
      <td>category</td>
    </tr>
    <tr>
      <td>surface_tp_train.ta</td>
      <td>1시간 평균 기온</td>
      <td>numeric</td>
    </tr>
    <tr>
      <td>surface_tp_train.td</td>
      <td>1시간 평균 이슬점온도</td>
      <td>numeric</td>
    </tr>
    <tr>
      <td>surface_tp_train.hm</td>
      <td>1시간 평균 상대 습도</td>
      <td>numeric</td>
    </tr>
    <tr>
      <td>surface_tp_train.ws</td>
      <td>1시간 평균 풍속</td>
      <td>numeric</td>
    </tr>
    <tr>
      <td>surface_tp_train.rn</td>
      <td>1시간 누적 강수량</td>
      <td>numeric</td>
    </tr>
    <tr>
      <td>surface_tp_train.re</td>
      <td>1시간 누적 강수 유무</td>
      <td>category</td>
    </tr>
    <tr>
      <td>surface_tp_train.ww</td>
      <td>현천개 현천</td>
      <td>category</td>
    </tr>
    <tr>
      <td>surface_tp_train.ts</td>
      <td>1시간 평균 지면온도</td>
      <td>target</td>
    </tr>
    <tr>
      <td>hh</td>
      <td>시간</td>
      <td>category</td>
    </tr>
    <tr>
      <td>dd</td>
      <td>날짜</td>
      <td>category</td>
    </tr>
    <tr>
      <td>mm</td>
      <td>달</td>
      <td>category</td>
    </tr>
    <tr>
      <td>new_year</td>
      <td>연도</td>
      <td>category</td>
    </tr>
    <tr>
      <td>date</td>
      <td>총 시간 datetime</td>
      <td>category</td>
    </tr>
  </table>
</div>


In [ ]:
## 지역 개수 10개를 병렬적으로 학습하게 넣을 예정
## window size 지정 필요

## 들어가는 데이터 열의 개수는

"""
'surface_tp_train_stn', 'surface_tp_train_ta', 'surface_tp_train_td',
       'surface_tp_train_hm', 'surface_tp_train_ws', 'surface_tp_train_rn',
       'surface_tp_train_re', 'surface_tp_train_ww', 'surface_tp_train_ts','time_idx'
       에서 target값인 surface_tp_train_ts, groupby pivot인 surface_tp_train_stn 를 제거하고 넣을 예정
       :8가지
"""
## model import
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
import datetime
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# GPU setting
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(0)) #Google colab = TESLA T4

# Training/Test ratio = 8:2

Train_ratio = 0.8
Test_ratio = 0.2

def labelencoding(data):
  x = data[['surface_tp_train_stn', 'surface_tp_train_ta', 'surface_tp_train_td',
        'surface_tp_train_hm', 'surface_tp_train_ws', 'surface_tp_train_rn',
        'surface_tp_train_re', 'surface_tp_train_ww', 'time_idx']]
  y = data["surface_tp_train_ts"]


  ## x에서 문자열 라벨 인코딩 진행:
  le = LabelEncoder()
  le.fit(x.surface_tp_train_ww)
  x.surface_tp_train_ww = le.transform(x.surface_tp_train_ww)
  return x, y

x_region_1_spring, y_region_1_spring = labelencoding(train_region_1_spring)
x_region_2_spring, y_region_2_spring = labelencoding(train_region_2_spring)
x_region_3_spring, y_region_3_spring = labelencoding(train_region_3_spring)
x_region_4_spring, y_region_4_spring = labelencoding(train_region_4_spring)
x_region_5_spring, y_region_5_spring = labelencoding(train_region_5_spring)
x_region_6_spring, y_region_6_spring = labelencoding(train_region_6_spring)
x_region_7_spring, y_region_7_spring = labelencoding(train_region_7_spring)
x_region_8_spring, y_region_8_spring = labelencoding(train_region_8_spring)
x_region_9_spring, y_region_9_spring = labelencoding(train_region_9_spring)
x_region_10_spring, y_region_10_spring = labelencoding(train_region_10_spring)

In [ ]:
def before_modeling(x, y, scaler = 'minmax'):
  # Data Division
  train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=Test_ratio, random_state=42, shuffle = False)

  ## scaling:
  # 카테고리 변수 지정
  cat = ["surface_tp_train_re", "surface_tp_train_ww", "surface_tp_train_stn", "time_idx"]

  for category in cat:
    train_x[category] = train_x[category].astype("category")

  numeric_cols = [col for col in train_x.columns if col not in cat]

  if scaler == 'minmax':
  # Normalizing , 둘다 학습하여 성능비교할것
    minmax = MinMaxScaler()
    train_y = train_y.values.reshape(-1, 1)
    test_y = test_y.values.reshape(-1, 1)
    minmax.fit(train_x[numeric_cols])
    train_x[numeric_cols]= minmax.transform(train_x[numeric_cols])
    test_x[numeric_cols] = minmax.transform(test_x[numeric_cols])

  elif scaler == 'standard':
    standard = StandardScaler()
    train_y = train_y.values.reshape(-1, 1)
    test_y = test_y.values.reshape(-1, 1)
    standard.fit(train_x[numeric_cols])
    train_x[numeric_cols]= standard.transform(train_x[numeric_cols])
    test_x[numeric_cols] = standard.transform(test_x[numeric_cols])

  elif scaler == 'robust':
    robust = RobustScaler()
    train_y = train_y.values.reshape(-1, 1)
    test_y = test_y.values.reshape(-1, 1)
    robust.fit(train_x[numeric_cols])
    train_x[numeric_cols]= robust.transform(train_x[numeric_cols])
    test_x[numeric_cols] = robust.transform(test_x[numeric_cols])

  # Check Data pre-processing
  print("Training shape : ", train_x.shape, train_y.shape)
  print("Test shape : ",test_x.shape, test_y.shape)
  return train_x, test_x, train_y, test_y

In [ ]:
## standard scaling 진행
train_x_region_1_spring, test_x_region_1_spring, train_y_region_1_spring, test_y_region_1_spring = before_modeling(x_region_1_spring, y_region_1_spring, 'robust')
train_x_region_2_spring, test_x_region_2_spring, train_y_region_2_spring, test_y_region_2_spring = before_modeling(x_region_2_spring, y_region_2_spring, 'robust')
train_x_region_3_spring, test_x_region_3_spring, train_y_region_3_spring, test_y_region_3_spring = before_modeling(x_region_3_spring, y_region_3_spring, 'robust')
train_x_region_4_spring, test_x_region_4_spring, train_y_region_4_spring, test_y_region_4_spring = before_modeling(x_region_4_spring, y_region_4_spring, 'robust')
train_x_region_5_spring, test_x_region_5_spring, train_y_region_5_spring, test_y_region_5_spring = before_modeling(x_region_5_spring, y_region_5_spring, 'robust')
train_x_region_6_spring, test_x_region_6_spring, train_y_region_6_spring, test_y_region_6_spring = before_modeling(x_region_6_spring, y_region_6_spring, 'robust')
train_x_region_7_spring, test_x_region_7_spring, train_y_region_7_spring, test_y_region_7_spring = before_modeling(x_region_1_spring, y_region_7_spring, 'robust')
train_x_region_8_spring, test_x_region_8_spring, train_y_region_8_spring, test_y_region_8_spring = before_modeling(x_region_8_spring, y_region_8_spring, 'robust')
train_x_region_9_spring, test_x_region_9_spring, train_y_region_9_spring, test_y_region_9_spring = before_modeling(x_region_9_spring, y_region_9_spring, 'robust')
train_x_region_10_spring, test_x_region_10_spring, train_y_region_10_spring, test_y_region_10_spring = before_modeling(x_region_10_spring, y_region_10_spring, 'robust')

In [ ]:
train_x_name = [train_x_region_1_spring, train_x_region_2_spring, train_x_region_3_spring, train_x_region_4_spring, train_x_region_5_spring,
                train_x_region_6_spring, train_x_region_7_spring, train_x_region_8_spring, train_x_region_9_spring, train_x_region_10_spring]

test_x_name = [test_x_region_1_spring, test_x_region_2_spring, test_x_region_3_spring, test_x_region_4_spring, test_x_region_5_spring,
                test_x_region_6_spring, test_x_region_7_spring, test_x_region_8_spring, test_x_region_9_spring, test_x_region_10_spring]

train_y_name = [train_y_region_1_spring, train_y_region_2_spring, train_y_region_3_spring, train_y_region_4_spring, train_y_region_5_spring,
                train_y_region_6_spring, train_y_region_7_spring, train_y_region_8_spring, train_y_region_9_spring, train_y_region_10_spring]

test_y_name = [test_y_region_1_spring, test_y_region_2_spring, test_y_region_3_spring, test_y_region_4_spring, test_y_region_5_spring,
                test_y_region_6_spring, test_y_region_7_spring, test_y_region_8_spring, test_y_region_9_spring, test_y_region_10_spring]

num = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


# # 비어있는 torch에 x 변수들 넣기
# def dataload(x_name, num, x_parallel):
#   ## train_x 변수 명에서 숫자가 몇인지를 보고, 이를 통해서 torch 위치를 지정하자
#   region_index = num
#   ## train_x 리스트로 제작:
#   columns_feature = ["surface_tp_train_ta", "surface_tp_train_td", "surface_tp_train_hm", "surface_tp_train_ws", "surface_tp_train_rn", "surface_tp_train_re", "surface_tp_train_ww"]
#   for i in range(len(x_name)):
#     list_feature = x_name[columns_feature].iloc[i].to_list()
#     ## 제작된 리스트를 순서대로 torch에 넣는다.
#     for j in range(len(list_feature)):
#       x_parallel[region_index, i, j] = list_feature[j]

## torch에 x 담기
# 비어있는 torch 생성
def empty_torch(train_x):
  num_power = 10
  num_features = 7 ## surface_tp_train_stn, time_idx 제외하고 리스트로 들어갈 예정
  length = int(train_x.shape[0])
  size = (length, num_power, num_features)## 데이터 개수, 지점 개수, 피처 개수
  train_x_parallel = torch.empty(size)
  return train_x_parallel


# 비어있는 torch에 x 변수들 넣기
def dataload(x_name, num, x_parallel):
  ## train_x 변수 명에서 숫자가 몇인지를 보고, 이를 통해서 torch 위치를 지정하자
  region_index = num
  ## train_x 리스트로 제작:
  columns_feature = ["surface_tp_train_ta", "surface_tp_train_td", "surface_tp_train_hm", "surface_tp_train_ws", "surface_tp_train_rn", "surface_tp_train_re", "surface_tp_train_ww"]
  for i in range(len(x_name)):
    list_feature = x_name[columns_feature].iloc[i].to_list() ## 변수 종류
    ## 제작된 리스트를 순서대로 torch에 넣는다.
    for j in range(len(list_feature)):
      x_parallel[i, region_index, j] = list_feature[j]



## torch_y 담기...
# 비어있는 torch 생성
def empty_target(train_x):
  num_power = 10 ## 지점수
  num_features = 1 ## surface_tp_train_stn, time_idx 제외하고 리스트로 들어갈 예정
  length = int(train_x.shape[0])
  size = (length, num_power, num_features)
  train_y_parallel = torch.empty(size)
  return train_y_parallel

# 비어있는 torch에 target 넣기
def target_torch(y_name, num, y_parallel):
  ## train_x 변수 명에서 숫자가 몇인지를 보고, 이를 통해서 torch 위치를 지정하자
  region_index = num
  ## train_x 리스트로 제작:
  for i in range(len(y_name)):
    tensor = y_name[i]
    ## 제작된 리스트를 순서대로 torch에 넣는다.
    for j in range(len(tensor)):
      y_parallel[i, region_index, j] = tensor[j]

In [ ]:
## train_x 병렬화
train_x_parallel = empty_torch(train_x_region_1_spring)
for num in range(0,10,1):
  train_x = train_x_name[num]
  num = num
  dataload(train_x, num, train_x_parallel)

In [ ]:
train_x_parallel

tensor([[[-0.6800, -0.4819, -0.0712,  ...,  0.0000,  0.0000,  3.0000],
         [-0.8143, -0.4337,  0.1261,  ...,  0.0000,  0.0000,  0.0000],
         [-1.0870, -0.4333,  0.4135,  ...,  0.0000,  0.0000,  3.0000],
         ...,
         [-1.0152, -0.7816, -0.1880,  ...,  0.0000,  0.0000,  3.0000],
         [-1.0152, -0.7816, -0.1880,  ...,  0.0000,  0.0000,  3.0000],
         [-1.0189, -0.7342, -0.2109,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.9067, -0.4458,  0.1966,  ...,  0.0000,  0.0000,  3.0000],
         [-0.9143, -0.6145,  0.0176,  ...,  0.0000,  0.0000,  0.0000],
         [-1.1739, -0.4222,  0.5064,  ...,  0.0000,  0.0000,  3.0000],
         ...,
         [-1.0909, -0.7241, -0.0513,  ...,  0.0000,  0.0000,  3.0000],
         [-1.0909, -0.7241, -0.0513,  ...,  0.0000,  0.0000,  3.0000],
         [-1.0755, -0.7089, -0.1211,  ...,  0.0000,  0.0000,  0.0000]],

        [[-1.0267, -0.4458,  0.3333,  ...,  0.0000,  0.0000,  3.0000],
         [-1.0143, -0.7349, -0.0293,  ...,  0

In [ ]:
## test_x 병렬화
test_x_parallel = empty_torch(test_x_region_1_spring)
for num in range(0,10,1):
  test_x = test_x_name[num]
  num = num
  dataload(test_x, num, test_x_parallel)

In [ ]:
test_x_parallel

tensor([[[-1.4800e+00, -8.7952e-01,  2.9345e-01,  ...,  0.0000e+00,
           1.0000e+00,  4.0000e+00],
         [-1.6429e+00, -8.6747e-01,  4.6921e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-1.7391e+00, -8.3333e-01,  5.4487e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [-1.9242e+00, -1.2759e+00,  3.1339e-02,  ...,  0.0000e+00,
           1.9000e+01,  0.0000e+00],
         [-1.9242e+00, -1.2759e+00,  3.1339e-02,  ...,  0.0000e+00,
           1.9000e+01,  0.0000e+00],
         [-1.9245e+00, -1.1139e+00,  1.4453e-01,  ...,  5.1000e+00,
           6.0000e+01,  4.0000e+00]],

        [[-1.4933e+00, -8.5542e-01,  3.4188e-01,  ...,  0.0000e+00,
           0.0000e+00,  3.0000e+00],
         [-1.6429e+00, -8.7952e-01,  4.7214e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         [-1.7681e+00, -8.4444e-01,  5.5449e-01,  ...,  0.0000e+00,
           0.0000e+00,  0.0000e+00],
         ...,
         [-1.9545e+00, -1

In [ ]:
## train_y 병렬화
train_y_parallel = empty_target(train_y_region_1_spring)
for num in range(0,10,1):
  train_y = train_y_name[num]
  num = num
  target_torch(train_y, num, train_y_parallel)

In [ ]:
train_y_parallel

tensor([[[17.1000],
         [15.3000],
         [12.8000],
         ...,
         [14.7000],
         [14.7000],
         [14.5000]],

        [[16.4000],
         [14.4000],
         [12.0000],
         ...,
         [14.2000],
         [14.2000],
         [14.3000]],

        [[15.9000],
         [13.6000],
         [11.7000],
         ...,
         [13.8000],
         [13.8000],
         [13.8000]],

        ...,

        [[26.3000],
         [25.8000],
         [25.5000],
         ...,
         [26.9000],
         [26.9000],
         [25.6000]],

        [[26.1000],
         [25.3000],
         [25.4000],
         ...,
         [26.4000],
         [26.4000],
         [25.2000]],

        [[25.6000],
         [25.0000],
         [25.3000],
         ...,
         [26.0000],
         [26.0000],
         [24.9000]]])

In [ ]:
## test_y 병렬화
test_y_parallel = empty_target(test_y_region_1_spring)
for num in range(0,10,1):
  test_y = test_y_name[num]
  num = num
  target_torch(test_y, num, test_y_parallel)

In [ ]:
test_y_parallel

tensor([[[12.4000],
         [11.3000],
         [11.4000],
         ...,
         [13.0000],
         [13.0000],
         [11.8000]],

        [[12.2000],
         [10.8000],
         [11.3000],
         ...,
         [12.9000],
         [12.9000],
         [11.5000]],

        [[11.9000],
         [10.6000],
         [11.1000],
         ...,
         [12.7000],
         [12.7000],
         [11.1000]],

        ...,

        [[29.2000],
         [30.5000],
         [29.3000],
         ...,
         [28.5000],
         [28.5000],
         [28.3000]],

        [[28.3000],
         [29.2000],
         [28.8000],
         ...,
         [27.8000],
         [27.8000],
         [27.9000]],

        [[27.6000],
         [28.9000],
         [28.3000],
         ...,
         [27.4000],
         [27.4000],
         [27.6000]]])

In [ ]:
train_x_parallel = Variable(train_x_parallel)
test_x_parallel = Variable(test_x_parallel)
train_y_parallel = Variable(train_y_parallel)
test_y_parallel = Variable(test_y_parallel)

In [ ]:
train_x_parallel

tensor([[[-0.6800, -0.4819, -0.0712,  ...,  0.0000,  0.0000,  3.0000],
         [-0.8143, -0.4337,  0.1261,  ...,  0.0000,  0.0000,  0.0000],
         [-1.0870, -0.4333,  0.4135,  ...,  0.0000,  0.0000,  3.0000],
         ...,
         [-1.0152, -0.7816, -0.1880,  ...,  0.0000,  0.0000,  3.0000],
         [-1.0152, -0.7816, -0.1880,  ...,  0.0000,  0.0000,  3.0000],
         [-1.0189, -0.7342, -0.2109,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.9067, -0.4458,  0.1966,  ...,  0.0000,  0.0000,  3.0000],
         [-0.9143, -0.6145,  0.0176,  ...,  0.0000,  0.0000,  0.0000],
         [-1.1739, -0.4222,  0.5064,  ...,  0.0000,  0.0000,  3.0000],
         ...,
         [-1.0909, -0.7241, -0.0513,  ...,  0.0000,  0.0000,  3.0000],
         [-1.0909, -0.7241, -0.0513,  ...,  0.0000,  0.0000,  3.0000],
         [-1.0755, -0.7089, -0.1211,  ...,  0.0000,  0.0000,  0.0000]],

        [[-1.0267, -0.4458,  0.3333,  ...,  0.0000,  0.0000,  3.0000],
         [-1.0143, -0.7349, -0.0293,  ...,  0

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

def mean_absolute_error(y_true, y_pred):
    return torch.mean(torch.abs(y_true - y_pred))

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.layer_1 = nn.Linear(hidden_size, 256)
        self.layer_2 = nn.Linear(256,256)
        self.layer_3 = nn.Linear(256,128)
        self.layer_out = nn.Linear(128, output_size)
        self.relu = nn.ReLU() #Activation Func
        self.tanh = nn.Tanh()  # Activation function




    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, (hn, cn) = self.lstm(x, (h0 , c0))
        out = self.layer_1(out) # first layer
        out = self.relu(out) # activation func relu
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.layer_3(out)
        out = self.relu(out)
        out = self.layer_out(out) #Output layer
        return out

# 입력과 출력 크기 설정
input_size = 7
hidden_size = 516
#hidden_size = 256 봄
num_layers = 2
output_size = 1

# 모델 인스턴스 생성
model = LSTMModel(input_size, hidden_size, num_layers, output_size)

# 손실 함수와 최적화 함수 설정
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# x 텐서와 y 텐서 생성 (임의의 값으로 초기화)
x_train = train_x_parallel.view(10*7,-1, 7) # (..., 40, 7)
y_train = train_y_parallel.view(10*7,-1, 1)  # (..., 40, 1)
x_valid = test_x_parallel.view(10*7,-1,7)  # (..., 40, 7)
y_valid = test_y_parallel.view(10*7, -1, 1)  # (..., 40, 1)

print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

# x_train

x_train.shape
# 학습
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        valid_outputs = model(x_valid)
        valid_loss = criterion(valid_outputs, y_valid)
        valid_mae = mean_absolute_error(valid_outputs, y_valid)

    if (epoch + 1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {loss.item():.4f}, Valid Loss: {valid_loss.item():.4f}, Valid MAE: {valid_mae.item():.4f}')

# 테스트
model.eval()
with torch.no_grad():
    predicted = model(x_valid)
    print("Predicted:", predicted)


torch.Size([10, 8832, 7])
torch.Size([10, 8832, 1])
torch.Size([10, 2208, 7])
torch.Size([10, 2208, 1])
Epoch [1/1000], Train Loss: 25.9989, Valid Loss: 25.2696, Valid MAE: 25.2696
Epoch [2/1000], Train Loss: 25.9647, Valid Loss: 25.2331, Valid MAE: 25.2331
Epoch [3/1000], Train Loss: 25.9275, Valid Loss: 25.1849, Valid MAE: 25.1849
Epoch [4/1000], Train Loss: 25.8779, Valid Loss: 25.1053, Valid MAE: 25.1053
Epoch [5/1000], Train Loss: 25.7939, Valid Loss: 24.9483, Valid MAE: 24.9483
Epoch [6/1000], Train Loss: 25.6257, Valid Loss: 24.6152, Valid MAE: 24.6152
Epoch [7/1000], Train Loss: 25.2658, Valid Loss: 23.8584, Valid MAE: 23.8584
Epoch [8/1000], Train Loss: 24.4647, Valid Loss: 22.3904, Valid MAE: 22.3904
Epoch [9/1000], Train Loss: 22.9910, Valid Loss: 20.2290, Valid MAE: 20.2290
Epoch [10/1000], Train Loss: 20.8394, Valid Loss: 17.1284, Valid MAE: 17.1284
Epoch [11/1000], Train Loss: 17.8632, Valid Loss: 13.6246, Valid MAE: 13.6246
Epoch [12/1000], Train Loss: 14.3375, Valid Los